****Markov Decision Process Model****

Assume a 3x3 Grid with states 0,1,2,3,4,5,6 - the bottom two spaces of the grid are blocked off, and the goal is to go from state 0 (the top left corner), to state 6 (the bottom right corner) in as few steps as possible. Navigating through state 2 yields a penalty, and navigating through state 6 yields a reward. In the stochastic case, our navigator has a 70% chance of travelling in the direction he/she intends, and a 30% chance he/she will navigate in any other direction that is allowed (equally weighted across alternative directions)


In [165]:
import numpy as np
import pandas as pd

In [166]:
#Formalization of problem
states = [0,1,2,3,4,5,6]
actions = {0: {'right':1, 'down':3}, 
           1: {'left':0, 'right':2, 'down':4}, 
           2: {'left':1, 'down':5}, 
           3: {'up':0, 'right':4}, 
           4: {'up':1, 'left':3, 'right':5},
           5: {'left':4, 'up':2, 'down':6},
           6: {}
          }
rewards = {0:0, 1:0, 2:-5, 3:0, 4:0, 5:0, 6:10}
discount = 0.9

In [167]:
def get_reward(state, next_state):
    if next_state in actions[state].values(): return rewards[next_state]
def state_exists(state, action):
    if action in actions[state].keys(): return True

In [172]:
#Bellman Equations + Value Iteration--> Deterministic Case
V = [0,0,0,0,0,0,0]
for i in V:
    V[6] = 0
    V[5] = max(10 + discount*V[6], discount*V[4], -5 + discount*V[2])
    V[4] = max(discount*V[1], discount*V[3], discount*V[5])
    V[3] = max(discount*V[0], discount*V[4]) 
    V[2] = max(discount*V[1], discount*V[5])
    V[1] = max(discount*V[0], -5 + discount*V[2], discount*V[4])
    V[0] = max(discount*V[1], discount*V[3])
pd.DataFrame(data = np.matrix(V), columns = ['0','1','2','3','4','5','6'])


,0,1,2,3,4,5,6
0,7.29,8.1,9.0,8.1,9.0,10.0,0.0


In [173]:
#Q Matrix --> Deterministic Case
Q_mat = pd.DataFrame(columns = ['up', 'right', 'down', 'left'])
for i in range(0,7,1):
    for j in Q_mat.columns:
        if state_exists(i,j):
            Q_mat.loc[i,j] = rewards[actions[i][j]] + discount*V[actions[i][j]]
        else: 
            Q_mat.loc[i,j] = 0
print("Optimal decision in 0 = right or down")
print("Optimal decision in 1 = down")
print("Optimal decision in 2 = down")
print("Optimal decision in 3 = right")
print("Optimal decision in 4 = right")
print("Optimal decision in 5 = down")
Q_mat


Optimal decision in 0 = right or down
Optimal decision in 1 = down
Optimal decision in 2 = down
Optimal decision in 3 = right
Optimal decision in 4 = right
Optimal decision in 5 = down


,up,right,down,left
0,0,7.29,7.29,0
1,0,3.1,8.1,6.561
2,0,0,9,7.29
3,6.561,8.1,0,0
4,7.29,9,0,7.29
5,3.1,0,10,8.1
6,0,0,0,0


In [170]:
#Bellman Equations & Value Iteration -- Stochastic Case
def bell_val(discount):
    V_s = [0,0,0,0,0,0,0]
    for i in range(0,10,1):
        V_s[6] = 0
        V_s[5] = max(0.7*(10 + discount*V_s[6]) + 0.15*(-5 + discount*V_s[2]) + 0.15*(discount*V_s[4])
               , 0.15*(10 + discount*V_s[6]) + 0.7*(-5 + discount*V_s[2]) + 0.15*(discount*V_s[4])
               , 0.15*(10 + discount*V_s[6]) + 0.15*(-5 + discount*V_s[2]) + 0.7*(discount*V_s[4]))
        V_s[4] = max(0.7*(discount*V_s[1]) + 0.15*(discount*V_s[5]) + 0.15*(discount*V_s[3])
               , 0.15*(discount*V_s[1]) + 0.7*(discount*V_s[5]) + 0.15*(discount*V_s[3])
               , 0.15*(discount*V_s[1]) + 0.15*(discount*V_s[5]) + 0.7*(discount*V_s[3]))
        V_s[3] = max(0.7*(discount*V_s[0]) + 0.3*(discount*V_s[4])
               , 0.3*(discount*V_s[0]) + 0.7*(discount*V_s[4]))
        V_s[2] = max(0.7*(discount*V_s[5]) + 0.3*(discount*V_s[1])
               , 0.3*(discount*V_s[5]) + 0.7*(discount*V_s[1]))
        V_s[1] = max(0.7*(-5 + discount*V_s[2]) + 0.15*(discount*V_s[4]) + 0.15*(discount*V_s[0])
               , 0.15*(-5 + discount*V_s[2]) + 0.7*(discount*V_s[4]) + 0.15*(discount*V_s[0])
               , 0.15*(-5 + discount*V_s[2]) + 0.15*(discount*V_s[4]) + 0.7*(discount*V_s[0]))
        V_s[0] = max(0.7*(discount*V_s[1]) + 0.3*(discount*V_s[3])
               , 0.3*(discount*V_s[1]) + 0.7*(discount*V_s[3]))
    return V_s

bell_val(0.9)

[4.574173320098797,
 4.719446155592721,
 6.285406441199981,
 5.237972790617084,
 6.355332986419627,
 7.955661789918901,
 0]

In [171]:
#Q_Matrix -- Stochastic Case
def q_stochastic(discount):
    Q_s = pd.DataFrame(columns = ['up', 'right', 'down', 'left'])
    for i in range(0,7,1):
        for j in Q_s.columns:
            if state_exists(i, j):
                summation = 0
                next_s = actions[i][j]
                summation += 0.7*(get_reward(i, next_s) + discount*V_s[next_s])
                possible_states = actions[i].values()
                if (len(possible_states) > 1):
                    for k in possible_states:
                        if k != next_s:
                            summation += 0.3/(len(possible_states) - 1) * (get_reward(i, k) + discount*V_s[k])
                Q_s.loc[i,j]= summation
            else:
                Q_s.loc[i,j] = 0
    print("Optimal decision in 0 = down")
    print("Optimal decision in 1 = down")
    print("Optimal decision in 2 = down")
    print("Optimal decision in 3 = right")
    print("Optimal decision in 4 = right")
    print("Optimal decision in 6 = down")
    
    return Q_s
q_stochastic(0.9)


Optimal decision in 0 = down
Optimal decision in 1 = down
Optimal decision in 2 = down
Optimal decision in 3 = right
Optimal decision in 4 = right
Optimal decision in 6 = down


,up,right,down,left
0,0,4.3875,4.57417,0
1,0,1.93529,4.7199,3.83823
2,0,0,6.28632,5.12128
3,4.59767,5.23889,0,0
4,4.75439,6.35632,0,5.01106
5,2.81778,0,7.9565,5.60239
6,0,0,0,0
